# Capstone Project - The Battle of Neighborhoods
This Notebook is the Final Assigment of the Course "Applied Data Science Capstone".



## Introduction/Business Problem
An investment group wants to set up a chain of gyms with a revolutionary concept in Andalusia. As part of their initial plan they want to locate their first headquarters in Seville (the capital) in a place where there are many gym users to be able to attract them.


## Data section
We need to collect data from all gyms in the city of Seville, including their name, identification, location (address, latitude, longitude) and then search for the "hot" neighbor where most of the place is located. For the purpose of data assets, we use FourSquare and apply folio to visualize a particular neighbor in which we will observe customer "traffic" and predict the proper location of the new gym in the city.

For that purpose we will use the Request "GET https://api.foursquare.com/v2/venues/explore" with the "query": "Gym".
Finally the searched location will appear on the folio map with the name "Lugar Buscado".


In [37]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [38]:
CLIENT_ID = 'W40WT0A01CMJTJDOXXSBLPHXMMR3EELM1EPXH21OLHEPDGG5' # your Foursquare ID
CLIENT_SECRET = 'AMRDWO5R3EBG0RNFWTBMYUTRMKOR4ATHZLITBVMYK4TGD2IS' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W40WT0A01CMJTJDOXXSBLPHXMMR3EELM1EPXH21OLHEPDGG5
CLIENT_SECRET:AMRDWO5R3EBG0RNFWTBMYUTRMKOR4ATHZLITBVMYK4TGD2IS


In [39]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "query": "Gym",
    "near": "Sevilla",
    "radius": 10000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [40]:
print(data)

<Response [200]>


In [41]:
d = data.json()["response"]
d.keys()

dict_keys(['geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [42]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Seville', 'Seville')

In [43]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 37.416723246192866, 'lng': -5.916579893727589},
  'sw': {'lat': 37.329702771734425, 'lng': -6.068793546063763}},
 64)

In [44]:
d["geocode"]

{'what': '',
 'where': 'sevilla',
 'center': {'lat': 37.38283, 'lng': -5.97317},
 'displayString': 'Seville, Andalusia, Spain',
 'cc': 'ES',
 'geometry': {'bounds': {'ne': {'lat': 37.45292999489887,
    'lng': -5.818495000128287},
   'sw': {'lat': 37.31276803496303, 'lng': -6.028492969509386}}},
 'slug': 'sevilla-spain',
 'longId': '72057594040438847'}

In [45]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [46]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [47]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b6021f5f964a520cfd629e3',
  'name': 'Club Metropolitan',
  'location': {'address': 'Av. de Eduardo Dato, 49',
   'lat': 37.383450698419296,
   'lng': -5.975260545828134,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.383450698419296,
     'lng': -5.975260545828134}],
   'postalCode': '41018',
   'cc': 'ES',
   'city': 'Sevilla',
   'state': 'Andalucía',
   'country': 'España',
   'formattedAddress': ['Av. de Eduardo Dato, 49',
    '41018 Sevilla Andalucía',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d175941735',
    'name': 'Gym / Fitness Center',
    'pluralName': 'Gyms or Fitness Centers',
    'shortName': 'Gym / Fitness',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId':

In [48]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4e5d3468483b18585970ec10',
  'name': 'Gimnasio Sevilla Gym',
  'location': {'address': 'Amor de Dios, 35',
   'lat': 37.39587849582059,
   'lng': -5.994382074570208,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.39587849582059,
     'lng': -5.994382074570208}],
   'postalCode': '41002',
   'cc': 'ES',
   'city': 'Sevilla',
   'state': 'Andalucía',
   'country': 'España',
   'formattedAddress': ['Amor de Dios, 35',
    '41002 Sevilla Andalucía',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d175941735',
    'name': 'Gym / Fitness Center',
    'pluralName': 'Gyms or Fitness Centers',
    'shortName': 'Gym / Fitness',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4e5d346

In [49]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i Gym" % len(df))
df.head()

{'address': 'Av. de Eduardo Dato, 49', 'lat': 37.383450698419296, 'lng': -5.975260545828134, 'labeledLatLngs': [{'label': 'display', 'lat': 37.383450698419296, 'lng': -5.975260545828134}], 'postalCode': '41018', 'cc': 'ES', 'city': 'Sevilla', 'state': 'Andalucía', 'country': 'España', 'formattedAddress': ['Av. de Eduardo Dato, 49', '41018 Sevilla Andalucía', 'España']}
{'address': 'Amor de Dios, 35', 'lat': 37.39587849582059, 'lng': -5.994382074570208, 'labeledLatLngs': [{'label': 'display', 'lat': 37.39587849582059, 'lng': -5.994382074570208}], 'postalCode': '41002', 'cc': 'ES', 'city': 'Sevilla', 'state': 'Andalucía', 'country': 'España', 'formattedAddress': ['Amor de Dios, 35', '41002 Sevilla Andalucía', 'España']}
{'address': 'Avda. Alcalde Luis Uruñuela', 'crossStreet': 'Frente Palacio De Congresos', 'lat': 37.40612887778902, 'lng': -5.933991364266831, 'labeledLatLngs': [{'label': 'display', 'lat': 37.40612887778902, 'lng': -5.933991364266831}], 'postalCode': '41020', 'cc': 'ES', 

,uid,name,shortname,address,postalcode,lat,lng
0,4b6021f5f964a520cfd629e3,Club Metropolitan,Gym / Fitness,,41018,37.383451,-5.975261
1,4e5d3468483b18585970ec10,Gimnasio Sevilla Gym,Gym / Fitness,,41002,37.395878,-5.994382
2,4e9ea70c2c5b47f3f525b246,Lowfit,Gym,,41020,37.406129,-5.933991
3,4be2f274b02ec9b69a194ec0,Galisport,Gym / Fitness,,41013,37.368877,-5.981208
4,4b90c41cf964a520b59633e3,Cuesta Sport,Gym / Fitness,,41004,37.389561,-5.991601


In [50]:
Sevilla_center = d["geocode"]["center"]
Sevilla_center

{'lat': 37.38283, 'lng': -5.97317}

In [51]:
from folium import plugins


map_Sevilla = folium.Map(location=[37.38283, -5.97317], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Sevilla)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Sevilla.add_child(plugins.HeatMap(hm_data))

map_Sevilla

In [52]:
lat = 37.38283
lng = -5.97317
map_Sevilla = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Lugar Buscado",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Sevilla)
map_Sevilla